# Input file brief discription

This notebook includes the process of reading the traffic flow data of a traffic scenario, which are then used as the input to the machine learning algorithm. 

Traffic flow data are stored in a sqlite file, in the table "MISECT". Inside table MISECT, we write query to extract link (road section) based traffic information, i.e., traffic flow (vehicle/hr), vehicle counts, mean travel time, speed and density.

In this script, we first get the names of tables from the master table to see if table MISECT is included. Then, we write query within table MISECT to get link (road) based traffic information to extract data from a senario. 

This discription is for files that store the link information only. The sensor information is stored in another sqlite file and can be extracted in the same manner. 

In [23]:
cd C:\GitFiles

C:\GitFiles


In [24]:
import pandas as pd
import numpy as np
import sqlite3, os

In [25]:
sqlite_file='C:\GitFiles\TSS_SANDAG_NETWORK_V8_typicalDayLastestNetwork_8-10AM_0.sqlite' #database file


In [26]:
db=sqlite3.connect(sqlite_file) #connecting to sqlite file

In [27]:
tableListQuery="SELECT name FROM sqlite_master WHERE type='table' ORDER BY Name"
cursor = db.cursor() 
cursor.execute(tableListQuery)
tables = map(lambda t: t[0], cursor.fetchall())
print (tables) #get tables

In [28]:
db

In [30]:
columnsQuery = "PRAGMA table_info(%s)" % tables[0]
#This query is just for column info
cursor.execute(columnsQuery)
columns=cursor.fetchall()
#for column in columns:
#    print (column)
numberOfColumns = len(cursor.fetchall())

In [34]:
#We take a look at the table misect, which include the link stats data we need
sectionQuery="SELECT * FROM MISECT"
cursor.execute(sectionQuery)
rows = cursor.fetchall()#get all the data we need


In [46]:
nameQuery="SELECT * FROM MISECT"
cursor.execute(nameQuery)
rowName = map(lambda x: x[0], cursor.description)#the 
#rowName=cursor.fetchall()
print rowName

['did', 'oid', 'eid', 'sid', 'ent', 'flow', 'flow_D', 'count', 'count_D', 'input_flow', 'input_flow_D', 'input_count', 'input_count_D', 'ttime', 'ttime_D', 'dtime', 'dtime_D', 'dtimeVQ', 'dtimeVQ_D', 'dtimeTtime', 'dtimeTtime_D', 'speed', 'speed_D', 'spdh', 'spdh_D', 'flow_capacity', 'flow_capacity_D', 'density', 'density_D', 'qmean', 'qmean_D', 'qmax', 'qmax_D', 'qvmean', 'qvmean_D', 'qvmax', 'qvmax_D', 'qvnbvehs', 'travel', 'travel_D', 'traveltime', 'traveltime_D', 'lane_changes', 'lane_changes_D', 'stime', 'stime_D', 'nstops', 'nstops_D']


In [195]:
#We want to have information for all the vehcile type need sid=0 (all vehicles), and the first time interval
#ent=1, time interval
#check if time interval 1 equals to time interval 0

#we want to get data from the first interval (ent=1) and all vehicle information (sid=0)
stat=rows[(rows[:,3]==0) & (rows[:,4]==1)]#look at rows where 

#Anther way is to write query directly, i.e.,
#"SELECT flow,count,ttime,speed FROM MISECT WHERE sid=0 AND ent=1"

In [196]:
flowIdx=rowName.index('flow') #flow: mean flow (veh/h)
countIdx=rowName.index('count') #vehicle counts (veh)
ttimeIdx=rowName.index('ttime') #mean travel time (s)
speedIdx=rowName.index('speed')#mean speed (km/h)
densityIdx=rowName.index('density')#density (veh/km)
qmeanIdx=rowName.index('qmean')#mean queue length (veh)
sectionIdx=rowName.index('oid')#sectionId

In [ ]:
#make a np array that include the index of flow, count,etc...
statIdx=np.array([sectionIdx,flowIdx, countIdx,ttimeIdx,speedIdx,qmeanIdx])
#This can also be done through SQL query
nonNullSecIdx=np.where(stat[:,flowIdx]>0)#filer out the links with no travel flows

secStatRep=stat[:,nonNullSecIdx]
secStatRep.shape

Note, anther way is to write query directly, where is more simple and desirable

"SELECT flow,count,ttime,speed,density,qmean,oid FROM MISECT WHERE sid=0 AND ent=1 AND flow>0"

In [216]:
print rowName

['did', 'oid', 'eid', 'sid', 'ent', 'flow', 'flow_D', 'count', 'count_D', 'input_flow', 'input_flow_D', 'input_count', 'input_count_D', 'ttime', 'ttime_D', 'dtime', 'dtime_D', 'dtimeVQ', 'dtimeVQ_D', 'dtimeTtime', 'dtimeTtime_D', 'speed', 'speed_D', 'spdh', 'spdh_D', 'flow_capacity', 'flow_capacity_D', 'density', 'density_D', 'qmean', 'qmean_D', 'qmax', 'qmax_D', 'qvmean', 'qvmean_D', 'qvmax', 'qvmax_D', 'qvnbvehs', 'travel', 'travel_D', 'traveltime', 'traveltime_D', 'lane_changes', 'lane_changes_D', 'stime', 'stime_D', 'nstops', 'nstops_D']


In [238]:
saveFileName='LinkOutput_7.txt'
np.savetxt(saveFileName,secStatRep,fmt=['%i', '%i', '%i', '%f', '%f', '%f'])

In [226]:
#db.commit() #commit the changes
db.close() #close the database